# Resources

 - https://www.kaggle.com/c/elo-merchant-category-recommendation
 - https://www.kaggle.com/truocpham/feature-engineering-and-lightgbm-starter
 - https://www.kaggle.com/dkmerona/fork-of-embedding-nn-64-feature

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os
import gc

from fastai import *
from fastai.tabular import *

In [4]:
# !pip uninstall kaggle 
#/opt/anaconda3/bin/kaggle competitions download -c elo-merchant-category-recommendation -p ~/my_data/elo


In [5]:
path = Path("~/my_data/elo/")

In [33]:
%%time
train_df = pd.read_csv(path/'train.csv', parse_dates=["first_active_month"])
test_df = pd.read_csv(path/'test.csv', parse_dates=["first_active_month"])

CPU times: user 336 ms, sys: 16 ms, total: 352 ms
Wall time: 351 ms


In [34]:
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [35]:
test_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04-01,C_ID_0ab67a22ab,3,3,1
1,2017-01-01,C_ID_130fd0cbdd,2,3,0
2,2017-08-01,C_ID_b709037bc5,5,1,1
3,2017-12-01,C_ID_d27d835a9f,2,1,0
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1


In [38]:
# np.where(test_df['first_active_month'].isna())
# print(test_df.iloc[11578] )
test_df.iloc[11578] = "2017-08-01"

In [39]:
add_datepart(train_df, "first_active_month", drop=False)
add_datepart(test_df, "first_active_month", drop=False)

In [40]:
hist_trans = pd.read_csv(path/"historical_transactions.csv")
hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [41]:
hist_trans = pd.get_dummies(hist_trans, columns=['category_2', 'category_3'])
hist_trans['authorized_flag'] = hist_trans['authorized_flag'].map({'Y': 1, 'N': 0})
hist_trans['category_1'] = hist_trans['category_1'].map({'Y': 1, 'N': 0})
hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0,1,0,0
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,1,0,0,0,0,1,0,0
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,1,0,0,0,0,1,0,0
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,1,0,0,0,0,1,0,0


In [42]:
# unique val in categorial col. 
hist_cat_col = []
for col in hist_trans._get_numeric_data().columns:
    if hist_trans[col].nunique() <6: hist_cat_col.append(col)
hist_cat_col

['authorized_flag',
 'category_1',
 'category_2_1.0',
 'category_2_2.0',
 'category_2_3.0',
 'category_2_4.0',
 'category_2_5.0',
 'category_3_A',
 'category_3_B',
 'category_3_C']

In [43]:
def aggregate_transaction(trans, prefix):
    trans.loc[:, 'purchase_date'] = pd.DatetimeIndex(trans['purchase_date']).astype(np.int64)*1e-9
    
    agg_func = {
        'authorized_flag' : ['sum', 'mean'],
        'category_1': ['mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'month_lag': ['min', 'max']
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values ]
    agg_trans.reset_index(inplace=True)
    df = (trans.groupby('card_id')
         .size()
         .reset_index(name='{}transactions_count'.format(prefix)))
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    return agg_trans

In [44]:
merch_hist = aggregate_transaction(hist_trans, prefix='hist_')
merch_hist.head()

,card_id,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,...,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_month_lag_min,hist_month_lag_max
0,C_ID_00007093c1,149,114,0.765101,0.187919,0.000000,0.000000,0.805369,0.0,0.006711,...,1.507069,-0.728876,0.298141,192,1.288591,6,1,0.764900,-12,0
1,C_ID_0001238066,123,120,0.975610,0.016260,0.772358,0.000000,0.000000,0.0,0.162602,...,0.768095,-0.734887,0.190235,198,1.609756,10,-1,1.485637,-5,0
2,C_ID_0001506ef0,66,62,0.939394,0.000000,0.030303,0.000000,0.969697,0.0,0.000000,...,1.493545,-0.740491,0.472284,1,0.015152,1,0,0.123091,-13,0
3,C_ID_0001793786,216,189,0.875000,0.009259,0.050926,0.351852,0.069444,0.0,0.000000,...,4.554145,-0.745405,0.836046,5,0.023148,1,0,0.150723,-9,0
4,C_ID_000183fdda,144,137,0.951389,0.027778,0.048611,0.006944,0.909722,0.0,0.006944,...,2.764788,-0.737892,0.524948,264,1.833333,10,-1,2.118846,-6,0


In [45]:
import gc
try:
    del hist_trans
    gc.collect()
except NameError:
    pass

train_mdf = pd.merge(train_df, merch_hist, on='card_id',how='left')
test_mdf = pd.merge(test_df, merch_hist, on='card_id',how='left')
del merch_hist
gc.collect()

28

In [46]:
train_mdf.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active_monthYear,first_active_monthMonth,first_active_monthWeek,first_active_monthDay,...,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_month_lag_min,hist_month_lag_max
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2017,6,22,1,...,2.258395,-0.739395,0.212139,4,0.015385,1,0,0.123314,-8,0
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,2017,1,52,1,...,4.630299,-0.742400,0.384967,543,1.551429,10,-1,1.510777,-12,0
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,2016,8,31,1,...,-0.145847,-0.730138,0.087380,0,0.000000,0,0,0.000000,-13,0
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,2017,9,35,1,...,1.445596,-0.740897,0.261624,84,1.090909,3,-1,0.588974,-5,0
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,2017,11,44,1,...,7.193041,-0.746156,1.352094,182,1.368421,12,1,1.896862,-3,0


In [47]:
new_trans = pd.read_csv(path/"new_merchant_transactions.csv")
new_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [48]:
new_trans = pd.get_dummies(new_trans, columns=['category_2', 'category_3'])
new_trans['authorized_flag'] = new_trans['authorized_flag'].map({'Y': 1, 'N': 0})
new_trans['category_1'] = new_trans['category_1'].map({'Y': 1, 'N': 0})
new_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0
2,1,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0
3,1,C_ID_415bb3a509,-1,1,1,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,-1,8,0,0,0,0,0,0,1,0
4,1,C_ID_ef55cf8d4b,-1,1,1,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,-1,29,0,0,0,0,0,0,1,0


In [49]:
merch_new = aggregate_transaction(new_trans, prefix='new_')
try:
    del new_trans
    gc.collect()
except NameError:
    pass
train_mdf = pd.merge(train_mdf, merch_new, on='card_id',how='left')
test_mdf = pd.merge(test_mdf, merch_new, on='card_id',how='left')
try:
    del merch_new
    gc.collect()
except NameError:
    pass
gc.collect()
train_mdf.head().T

,0,1,2,3,4
first_active_month,2017-06-01 00:00:00,2017-01-01 00:00:00,2016-08-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
first_active_monthYear,2017,2017,2016,2017,2017
first_active_monthMonth,6,1,8,9,11
first_active_monthWeek,22,52,31,35,44
first_active_monthDay,1,1,1,1,1


In [50]:
cat_cols = []
cont_cols = []
for col in train_mdf.columns:
    if train_mdf[col].nunique() < 7:
        cat_cols.append(col)
    else:
        cont_cols.append(col)
    
print ("cat_cols\n{0}, \n cont_cols\n{1}".format(cat_cols, cont_cols))


cat_cols
['feature_1', 'feature_2', 'feature_3', 'first_active_monthDay', 'first_active_monthIs_month_end', 'first_active_monthIs_month_start', 'first_active_monthIs_quarter_end', 'first_active_monthIs_quarter_start', 'first_active_monthIs_year_end', 'first_active_monthIs_year_start', 'new_authorized_flag_mean', 'new_month_lag_min', 'new_month_lag_max'], 
 cont_cols
['first_active_month', 'card_id', 'target', 'first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek', 'first_active_monthDayofweek', 'first_active_monthDayofyear', 'first_active_monthElapsed', 'hist_transactions_count', 'hist_authorized_flag_sum', 'hist_authorized_flag_mean', 'hist_category_1_mean', 'hist_category_2_1.0_mean', 'hist_category_2_2.0_mean', 'hist_category_2_3.0_mean', 'hist_category_2_4.0_mean', 'hist_category_2_5.0_mean', 'hist_category_3_A_mean', 'hist_category_3_B_mean', 'hist_category_3_C_mean', 'hist_purchase_amount_sum', 'hist_purchase_amount_mean', 'hist_purchase_amount_max', 'hist_

In [51]:
dep_var = 'target'
procs = [FillMissing, Categorify, Normalize]

cont_cols = [ 'first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek', 'first_active_monthDayofweek', 'first_active_monthDayofyear', 'first_active_monthElapsed', 'hist_transactions_count', 'hist_authorized_flag_sum', 'hist_authorized_flag_mean', 'hist_category_1_mean', 'hist_category_2_1.0_mean', 'hist_category_2_2.0_mean', 'hist_category_2_3.0_mean', 'hist_category_2_4.0_mean', 'hist_category_2_5.0_mean', 'hist_category_3_A_mean', 'hist_category_3_B_mean', 'hist_category_3_C_mean', 'hist_purchase_amount_sum', 'hist_purchase_amount_mean', 'hist_purchase_amount_max', 'hist_purchase_amount_min', 'hist_purchase_amount_std', 'hist_installments_sum', 'hist_installments_mean', 'hist_installments_max', 'hist_installments_min', 'hist_installments_std', 'hist_month_lag_min', 'hist_month_lag_max', 'new_transactions_count', 'new_authorized_flag_sum', 'new_category_1_mean', 'new_category_2_1.0_mean', 'new_category_2_2.0_mean', 'new_category_2_3.0_mean', 'new_category_2_4.0_mean', 'new_category_2_5.0_mean', 'new_category_3_A_mean', 'new_category_3_B_mean', 'new_category_3_C_mean', 'new_purchase_amount_sum', 'new_purchase_amount_mean', 'new_purchase_amount_max', 'new_purchase_amount_min', 'new_purchase_amount_std', 'new_installments_sum', 'new_installments_mean', 'new_installments_max', 'new_installments_min', 'new_installments_std']

In [52]:
idx = np.where(test_mdf['first_active_monthYear'].isna())
test_mdf.loc[idx]['first_active_monthYear']= 2017

In [53]:
try:
    del data
    del learn
    gc.collect()
except Exception:
    pass
# use merged data frames 

data = (TabularList.from_df(train_mdf, path=path, cat_names=cat_cols, 
                            cont_names=cont_cols, procs=procs,)
    .random_split_by_pct()
    .label_from_df(cols=dep_var, label_cls=FloatList,log=False)
    .add_test(ItemList.from_df(test_mdf,path))
    .databunch())

Exception: There are nan values in field hist_transactions_count but there were none in the training set. 
                Please fix those manually.

In [ ]:
def rmse(pred:FloatTensor, targ:FloatTensor):
    "RMSE between `pred` and `targ`."
    assert pred.numel() == targ.numel(), "Expected same numbers of elements in pred & targ"
    if len(pred.shape)==2: pred=pred.squeeze(1)
    var = (targ - pred)
    return torch.sqrt((var**2).mean())

max_y = (np.max(train_mdf[dep_var])*1.2)
y_range = torch.tensor([0, max_y], device=defaults.device)
y_range

In [ ]:
learn = tabular_learner(data, layers=[1000, 500], ps=[0.001,0.01], emb_drop=0.04, 
                        model_dir='..', 
                        y_range=y_range, metrics=rmse)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_start=0, skip_end=-10)

In [ ]:
learn.fit_one_cycle(1, 3e-2,)

In [ ]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
from datetime import datetime
ids = test_df['card_id']
vals = np.exp(preds.numpy())
sub = pd.DataFrame({'card_id': ids, 'target': vals[:,0]})
sub.to_csv(path/("submission-%s.csv"%datetime.now().date()), index = False, header = True)
sub.head()

In [ ]:
np.max(sub), np.min(sub)

In [ ]:
np.min(train_mdf[dep_var])

In [54]:
np.where(test_mdf['hist_transactions_count'].isna())

(array([11578]),)

In [56]:
test_mdf.iloc[11578]

first_active_month                    2017-08-01 00:00:00
card_id                                        2017-08-01
feature_1                                      2017-08-01
feature_2                                      2017-08-01
feature_3                                      2017-08-01
first_active_monthYear                               2017
first_active_monthMonth                                 8
first_active_monthWeek                                 31
first_active_monthDay                                   1
first_active_monthDayofweek                             1
first_active_monthDayofyear                           213
first_active_monthIs_month_end                      False
first_active_monthIs_month_start                     True
first_active_monthIs_quarter_end                    False
first_active_monthIs_quarter_start                  False
first_active_monthIs_year_end                       False
first_active_monthIs_year_start                     False
first_active_m